# Calendar Generator - Hungary Detailed Calendar Features

## Overview

This notebook generates comprehensive calendar features for Hungary that are essential for electricity load forecasting. Calendar features capture regular patterns (daily, weekly, seasonal) and special events (holidays, weekends) that significantly influence electricity consumption.

### Feature Categories

1. **Cyclic Features**: Sinusoidal encodings for hour, day of week, month, day of year (captures cyclical patterns)
2. **Binary Features**: Flags for weekends, holidays, workdays, peak hours, school holidays, etc.
3. **One-Hot Encoded Features**: Day type (weekday, weekend, holiday, Monday), season indicators
4. **Special Events**: Bridge days, paydays, national holidays, Christmas period

### Temporal Resolution

- **Resolution**: 15-minute intervals
- **Timezone**: UTC (Coordinated Universal Time)
- **Period**: 2015-2024

### Hungary-Specific Features

- Public holidays for Hungary
- Business hours (8:00-18:00 local time)
- School holiday periods
- National day (August 20 - St. Stephen's Day)
- Payday patterns (typically mid-month and end-of-month)

These features help the model understand when electricity demand is likely to be higher or lower due to human activity patterns.

In [1]:
import sys
sys.path.append('../src')

from calendar_generator import HungaryCalendarGenerator
from datetime import datetime
import pandas as pd

In [2]:
!pip install holidays

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: C:\Users\Baturalp Taha YILMAZ\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
generator = HungaryCalendarGenerator()

print(f"Timezone: {generator.local_tz}")
print(f"Business Hours: {generator.business_hours_start:02d}:00 - {generator.business_hours_end:02d}:00 (Local Time)")

Timezone: Europe/Budapest
Business Hours: 08:00 - 18:00 (Local Time)


In [4]:
start_date = datetime(2015, 1, 1)
end_date = datetime(2024, 12, 31)

print(f"Takvim generateulacak tarih aralığı: {start_date.date()} - {end_date.date()}")

Takvim oluşturulacak tarih aralığı: 2015-01-01 - 2024-12-31


In [5]:
# Detaylı calendar özelliklerini generate
calendar_df = generator.generate_calendar(start_date, end_date)

2025-12-10 22:00:48,635 - INFO - Takvim oluşturuluyor: 350,688 kayıt
2025-12-10 22:00:52,416 - INFO - ✅ Takvim oluşturuldu: 350,688 kayıt, 45 özellik
2025-12-10 22:00:52,417 - INFO -    - Siklik özellikler: 10
2025-12-10 22:00:52,417 - INFO -    - Binary özellikler: 16
2025-12-10 22:00:52,417 - INFO -    - OHE özellikler: 8


In [6]:
print("\n📅 Takvim Verisi:")
print(calendar_df.head(10))
print(f"\nToplam rows sayısı: {len(calendar_df):,}")
print(f"Date range: {calendar_df['datetime'].min()} - {calendar_df['datetime'].max()}")
print(f"\nToplam sütun sayısı: {len(calendar_df.columns)}")
print(f"\nSütunlar: {list(calendar_df.columns)}")


📅 Takvim Verisi:
                   datetime  hour_sin  hour_cos    q_of_h_sin    q_of_h_cos  \
0 2015-01-01 00:00:00+00:00  0.258819  0.965926  0.000000e+00  1.000000e+00   
1 2015-01-01 00:15:00+00:00  0.258819  0.965926  1.000000e+00  6.123234e-17   
2 2015-01-01 00:30:00+00:00  0.258819  0.965926  1.224647e-16 -1.000000e+00   
3 2015-01-01 00:45:00+00:00  0.258819  0.965926 -1.000000e+00 -1.836970e-16   
4 2015-01-01 01:00:00+00:00  0.500000  0.866025  0.000000e+00  1.000000e+00   
5 2015-01-01 01:15:00+00:00  0.500000  0.866025  1.000000e+00  6.123234e-17   
6 2015-01-01 01:30:00+00:00  0.500000  0.866025  1.224647e-16 -1.000000e+00   
7 2015-01-01 01:45:00+00:00  0.500000  0.866025 -1.000000e+00 -1.836970e-16   
8 2015-01-01 02:00:00+00:00  0.707107  0.707107  0.000000e+00  1.000000e+00   
9 2015-01-01 02:15:00+00:00  0.707107  0.707107  1.000000e+00  6.123234e-17   

   weekday_sin  weekday_cos  month_sin  month_cos   day_sin  ...  \
0     0.433884    -0.900969        0.5   0.8

In [7]:
# Özellik istatistikleri
print("\n📊 Özellik İstatistikleri:")
print(f"\nHafta sonu günleri: {calendar_df['is_weekend'].sum():,} rows")
print(f"Holiday günleri: {calendar_df['is_holiday'].sum():,} rows")
print(f"İş günleri: {calendar_df['is_workday'].sum():,} rows")
print(f"İş saatleri: {calendar_df['is_work_hour'].sum():,} rows")
print(f"Öğle yemeği saatleri: {calendar_df['is_lunch_hour'].sum():,} rows")
print(f"Akşam piki saatleri: {calendar_df['is_peak_hour'].sum():,} rows")
print(f"Off-peak saatleri: {calendar_df['is_off_peak_hour'].sum():,} rows")
print(f"Shoulder saatleri: {calendar_df['is_shoulder_hour'].sum():,} rows")
print(f"Okul tatili günleri: {calendar_df['is_school_holiday'].sum():,} rows")
print(f"\nHoliday öncesi günler: {calendar_df['before_holiday_1day'].sum():,} rows")
print(f"Holiday sonrası günler: {calendar_df['after_holiday_1day'].sum():,} rows")
print(f"Bridge günler: {calendar_df['is_bridge_day'].sum():,} rows")
print(f"Ödeme günleri: {calendar_df['is_payday'].sum():,} rows")
print(f"Yılbaşı dönemi: {calendar_df['christmas_period'].sum():,} rows")
print(f"20 Ağustos (Aziz Stephen): {calendar_df['natl_day_peak_20Aug'].sum():,} rows")
print(f"\nDay Type OHE:")
day_type_cols = [col for col in calendar_df.columns if 'day_type_OHE' in col]
for col in day_type_cols:
    print(f"  {col}: {calendar_df[col].sum():,} rows")
print(f"\nSeason OHE:")
season_cols = [col for col in calendar_df.columns if 'season_flag_OHE' in col]
for col in season_cols:
    print(f"  {col}: {calendar_df[col].sum():,} rows")


📊 Özellik İstatistikleri:

Hafta sonu günleri: 100,224 kayıt
Tatil günleri: 14,392 kayıt
İş günleri: 240,192 kayıt
İş saatleri: 100,080 kayıt
Öğle yemeği saatleri: 20,016 kayıt
Akşam piki saatleri: 40,032 kayıt
Off-peak saatleri: 87,672 kayıt
Shoulder saatleri: 58,448 kayıt
Okul tatili günleri: 106,560 kayıt

Tatil öncesi günler: 4,128 kayıt
Tatil sonrası günler: 6,240 kayıt
Bridge günler: 192 kayıt
Ödeme günleri: 69,120 kayıt
Yılbaşı dönemi: 2,880 kayıt
20 Ağustos (Aziz Stephen): 960 kayıt

Day Type OHE:
  day_type_OHE_Holiday: 14,392 kayıt
  day_type_OHE_Monday: 45,984 kayıt
  day_type_OHE_Normal_Weekday: 194,208 kayıt
  day_type_OHE_Weekend: 96,104 kayıt

Season OHE:
  season_flag_OHE_Autumn: 87,400 kayıt
  season_flag_OHE_Spring: 88,280 kayıt
  season_flag_OHE_Summer: 88,320 kayıt
  season_flag_OHE_Winter: 86,688 kayıt


In [8]:
# Takvim datasini save
generator.save_csv(calendar_df, "../data/raw/hungary_calendar_2015_2024.csv")

print("\n✅ Takvim datasi saved!")

2025-12-10 22:00:58,120 - INFO - ✅ Takvim kaydedildi: ../data/raw/hungary_calendar_2015_2024.csv



✅ Takvim verisi kaydedildi!
